In [ ]:
import sys
sys.path.append("../../src/earthtext")

from osm import osm
import os
from progressbar import progressbar as pbar
import shapely as sh 
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)
import geopandas as gpd
import pandas as pd
from importlib import reload
from rlxutils import Command
import rlxutils
reload(osm)

In [2]:
orig_partsdir = "/opt/data/osm/california-parts/"
dest_partsdir = "/opt/data/california-worldcover-chips/osm"
os.makedirs(dest_partsdir, exist_ok=True)
chip_ids_pbf = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".pbf")]
chip_ids_geojson = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".geojson")]

if len(set(chip_ids_pbf).intersection(set(chip_ids_geojson)))!=len(chip_ids_pbf)!=len(chip_ids_geojson):
    raise ValueError("missing chips in geojson o pbf")

In [3]:
# load geojson

In [4]:
orig_grid = []
for chip_id in chip_ids_geojson:
    with open(f"{orig_partsdir}/{chip_id}.geojson") as f:
        orig_grid.append([chip_id, sh.from_geojson(f.read())])

orig_grid = gpd.GeoDataFrame(pd.DataFrame(orig_grid, columns=['chip_id', 'geometry']), crs=epsg4326)

In [5]:
orig_grid.head()

,chip_id,geometry
0,000235e893b24,"POLYGON ((-121.87715 39.14450, -121.65850 39.1..."
1,0002c308d55fc,"POLYGON ((-114.92112 35.05498, -114.92112 35.2..."
2,0018e8bbce095,"POLYGON ((-120.25595 36.95832, -120.55074 36.9..."
3,003fa84955512,"POLYGON ((-119.13010 35.78528, -119.40375 35.7..."
4,004aaa9df44f2,"POLYGON ((-124.14191 40.08352, -124.24849 40.0..."


In [6]:
dest_grid = gpd.read_file("/opt/data/california-worldcover-chips.fgb")
dest_grid.head()

,col,row,geometry
0,131584,119552,"POLYGON ((-114.01333 32.01600, -114.01333 32.0..."
1,131584,119296,"POLYGON ((-114.01333 32.03733, -114.01333 32.0..."
2,131328,119296,"POLYGON ((-114.03467 32.03733, -114.03467 32.0..."
3,131328,119552,"POLYGON ((-114.03467 32.01600, -114.03467 32.0..."
4,131072,119552,"POLYGON ((-114.05600 32.01600, -114.05600 32.0..."


In [15]:
chip_ids = ['0d0d59f364f72', '26017774321e9']
_orig_grid = orig_grid[orig_grid.chip_id.isin(chip_ids)]
_orig_grid.shape

(2, 2)

In [ ]:
c = _orig_grid.geometry.values[1]

dg = dest_grid[[i.intersects(c) for i in dest_grid.geometry.values]]
pd.concat([dg, _orig_grid]).explore()

In [16]:
_orig_grid.head()

,chip_id,geometry
214,0d0d59f364f72,"POLYGON ((-122.24843 37.22020, -122.24843 37.3..."
648,26017774321e9,"POLYGON ((-121.22865 36.97216, -121.47409 36.9..."


In [19]:
notfound = []
for count, (_, orig_row) in enumerate(_orig_grid.iterrows()):
    et = rlxutils.ElapsedTimes()
    orig_row_geom = orig_row.geometry
    dg = dest_grid[[i.intersects(orig_row_geom) for i in dest_grid.geometry.values]]
    print (f"{count}/{len(orig_grid)}", orig_row.chip_id, 'dest_chips', len(dg), flush=True)

    parquets_cache = {}
    for _, dest_row in pbar(dg.iterrows(), max_value=len(dg)):
        
        dest_row_geom = dest_row.geometry
        
        dest_parquet = f"{dest_partsdir}/{osm.get_region_hash(dest_row_geom)}.parquet"
        if os.path.isfile(dest_parquet):
            continue

        with et("ogintersect"):
            og = orig_grid[[i.intersects(dest_row_geom) for i in orig_grid.geometry.values]]
        
        if len(og) == 0:
            continue
            
        dgdata = []
        for chip_id in og.chip_id.values:
            if chip_id in parquets_cache.keys():
                p = parquets_cache[chip_id]
            else:
                with et("read_parquet"):
                    fname = f"{orig_partsdir}/{chip_id}.parquet"
                    if not os.path.isfile(fname):
                        notfound.append(fname)
                        continue
                    p = gpd.read_parquet(fname)
                    parquets_cache[chip_id]=p
            
            with et("dgintersect"):
                p = p[p.intersects(dest_row_geom)]
            dgdata.append(p)
        if len(dgdata)==0:
            continue
        dgdata = pd.concat(dgdata)
        with et("save"):
            dgdata.to_parquet(dest_parquet)
    #print(et)

0/984 0d0d59f364f72 dest_chips 126


100% (126 of 126) |######################| Elapsed Time: 0:00:11 Time:  0:00:110001


1/984 26017774321e9 dest_chips 108


100% (108 of 108) |######################| Elapsed Time: 0:00:00 Time:  0:00:000000


In [10]:
len(notfound)

154

In [12]:
np.unique(notfound)

array(['/opt/data/osm/california-parts//0d0d59f364f72.parquet',
       '/opt/data/osm/california-parts//26017774321e9.parquet'],
      dtype='<U53')

In [20]:
dgdata.head()

,tags,geometry,kind,length,area
7321872664,"{'ALAND': None, 'AREAID': None, 'AWATER': None...",POINT (-120.78318 37.58513),node,0.000000,0.000000
42,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.78301 37.60016, -120.77628 37...",way,1179.411007,0.000000
84,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.77621 37.60174, -120.77628 37...",way,228.290886,0.000000
93,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.78361 37.58751, -120.78303 37...",way,812.762487,0.000000
567,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","POLYGON ((-120.78083 37.58378, -120.78210 37.5...",way,785.161964,36789.620588


In [21]:
k = gpd.read_parquet(dest_parquet)
k.head()

,tags,geometry,kind,length,area
7321872664,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...",POINT (-120.78318 37.58513),node,0.000000,0.000000
42,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.78301 37.60016, -120.77628 37...",way,1179.411007,0.000000
84,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.77621 37.60174, -120.77628 37...",way,228.290886,0.000000
93,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.78361 37.58751, -120.78303 37...",way,812.762487,0.000000
567,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","POLYGON ((-120.78212 37.58410, -120.78313 37.5...",way,785.161964,36789.620588


In [29]:
k['area'].sum() - dgdata['area'].sum(), k['length'].sum() - dgdata['length'].sum()

(0.0, 0.0)

In [111]:
et = rlxutils.ElapsedTimes()
with et("0"):
    f = [i.intersects(dest_row_geom) for i in ogdata.geometry.values]
with et("0f"):
    f1 = ogdata.intersects(dest_row_geom).values
with et("1"):
    dgdata = ogdata[f]
with et("copy"):
    dgdata = dgdata.copy()
with et("2"):
    dgdata['geometry'] = [i.intersection(dest_row_geom) for i in dgdata.geometry.values]
et

{'0': 0.33873724937438965, '0f': 0.003275632858276367, '1': 0.0021512508392333984, 'copy': 0.00016117095947265625, '2': 0.003275156021118164}

In [115]:
(np.r_[f] == f1).mean()

1.0

In [102]:
i = ogdata.geometry.values[0]
et = rlxutils.ElapsedTimes()

with et(1):
    i.intersects(dest_row_geom)
with et(2):
    i.intersection(dest_row_geom)
et

{1: 6.079673767089844e-05, 2: 0.0003120899200439453}

In [107]:
ogdata.intersects(dest_row_geom).sum()

151

In [90]:
len(parquets)

2

In [91]:
dgdata

,tags,geometry,kind,length,area
925659355,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...",POINT (-120.81544 37.75681),node,0.000000,0.0
1313,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.79871 37.75713, -120.79796 37...",way,561.833488,0.0
2132,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.81727 37.75772, -120.81726 37...",way,3917.776839,0.0
2728,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.80706 37.75467, -120.80706 37...",way,1502.624837,0.0
5091,"{'ALAND': None, 'AREAID': None, 'AREA_12_13': ...","LINESTRING (-120.79733 37.75548, -120.79812 37...",way,1009.005534,0.0
...,...,...,...,...,...
7429,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.80711 37.76202, -120.80746 37...",way,49.309820,0.0
7430,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","MULTILINESTRING ((-120.80711 37.76202, -120.80...",way,246.565766,0.0
7431,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.80594 37.76187, -120.80593 37...",way,87.552198,0.0
7551,"{'ALAND': None, 'AREAID': None, 'AWATER': None...","LINESTRING (-120.81823 37.77231, -120.81799 37...",way,68.196587,0.0


In [44]:
dgdata = ogdata[[i.intersects(c) for i in ogdata.geometry.values]].copy()
dgdata['geometry'] = [i.intersection(c) for i in dgdata.geometry.values]

In [119]:
pd.concat([dgdata, gpd.GeoDataFrame([dest_row], crs=epsg4326)]).explore()

In [40]:
for _, dest_row in pbar(dest_grid[11000:].iterrows(), max_value = len(dest_grid)):

    c = dest_row.geometry
    
    og = orig_grid[[i.intersects(c) for i in orig_grid.geometry.values]]
    
    if len(og) == 0:
        continue
        
    dest_pbf = f"{dest_partsdir}/{osm.get_region_hash(c)}.pbf"

    parquets = []
    for chip_id in og.chip_id.values:
        if chip_id in parquets_cache.keys():
            parquets.append(parquets_cache[chip_id])
        else:
            p = gpd.read_parquet(f"{orig_partsdir}/{chip_id}.parquet")
            print (p.shape)
            parquets.append(p)
            if len(p)>200000:
                parquets_cache[chip_id]=p
                print ("cache size", len(parquets_cache), [len(i) for i in parquets_cache.values()])
        
    
    ogdata = pd.concat([gpd.read_parquet(f"{orig_partsdir}/{i}.parquet") for i in og.chip_id.values])
    break

  0% (0 of 111920) |                     | Elapsed Time: 0:00:00 ETA:  --:--:--

(4355, 5)
(4458, 5)


In [ ]:
k = gpd.GeoDataFrame([sh.geometry.box(-116.5877625,32.7143199,-116.5725234,32.7475016)], columns=['geometry'], crs=epsg4326)

pd.concat([ogdata.sample(1000), k, dest_grid.loc[11000:11001]]).explore()

In [ ]:
ogdata.sample(1000).explore()

In [340]:
list(parquets_cache.keys())

['1f56b681fda91', '1ac65d4496013']

In [341]:
parquets_cache['1f56b681fda91'].head()

,tags,geometry,kind,length,area
10537865,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.31868 34.19228),node,0.0,0.0
10537871,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.30324 34.18102),node,0.0,0.0
10537875,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.28975 34.17106),node,0.0,0.0
10683747,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.49486 34.32006),node,0.0,0.0
14758466,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.32173 34.18428),node,0.0,0.0


In [342]:
parquets_cache['1ac65d4496013'].head()

,tags,geometry,kind,length,area
10537865,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.31868 34.19228),node,0.0,0.0
10537871,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.30324 34.18102),node,0.0,0.0
10537875,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.28975 34.17106),node,0.0,0.0
10683747,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.49486 34.32006),node,0.0,0.0
14758466,"{'AIN': None, 'ALAND': None, 'AREAID': None, '...",POINT (-118.32173 34.18428),node,0.0,0.0
